In [1]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient('localhost', 27017)
collection = client['companies'].tableauproject

In [2]:
# hacemos una consulta para que nos dé toda la colección y lo convertimos en df
data = pd.DataFrame(list(collection.find()))
data.head()

,_id,category_code,city,country_code,founded_year,geopoint,latitude,longitude,name,number_of_employees,type
0,5cd453a1e74b56f50369c6e3,web,New York,USA,2005,"{'type': 'Point', 'coordinates': [-73.9964312,...",40.723731,-73.996431,Wetpaint,47,startup
1,5cd453a1e74b56f50369c6e4,games_video,San Francisco,USA,2006,"{'type': 'Point', 'coordinates': [-122.409173,...",37.788482,-122.409173,Kyte,40,startup
2,5cd453a1e74b56f50369c6e5,web,Pleasanton,USA,2005,"{'type': 'Point', 'coordinates': [-121.907768,...",37.697805,-121.907768,Jangl SMS,22,startup
3,5cd453a1e74b56f50369c6e6,games_video,Los Angeles,USA,2006,"{'type': 'Point', 'coordinates': [-118.254558,...",34.051409,-118.254558,Stickam,35,startup
4,5cd453a1e74b56f50369c6e7,games_video,San Bruno,USA,2005,"{'type': 'Point', 'coordinates': [-122.426804,...",37.627971,-122.426804,YouTube,0,startup


In [3]:
# función para sacar el número de empresas que hay en un radio específico de un geopoint concreto
def get_near_companies(geopoint, max_distance):
    nearcomps = pd.DataFrame(list(collection.find(
        {'geopoint': { '$near': {'$geometry': geopoint, '$maxDistance': max_distance}}}, {'name': 1})))
    return len(nearcomps)

In [4]:
# vamos a ver el número de startups y empresas grandes que tenemos en nuestro dataframe.
# vemos que hay solo 15 empresas grandes.
data['type'].value_counts()

startup          1662
large company      15
Name: type, dtype: int64

In [5]:
# creamos columna nueva con el número de empresas cerca de cada empresa
distances = [500, 2000]
for e in distances:
    data['companies{}'.format(e)] = data['geopoint'].apply(lambda x: get_near_companies(x, e))

In [6]:
# función para añadir una columna para que nos diga si en el radio indicado existe alguna empresa grande (True/False)
def get_near_large_company(geopoint, max_distance):
    nearcomps = pd.DataFrame(list(collection.find(
        {'geopoint': { '$near': {'$geometry': geopoint, '$maxDistance': max_distance}}}, {'type': 1})))
    if len(nearcomps[nearcomps['type'] == 'large company']) > 0: return True
    else: return False

In [7]:
for e in distances:
    data['large{}'.format(e)] = data['geopoint'].apply(lambda x: get_near_large_company(x, e))

In [8]:
# función para sacar el número de empleados que hay en un radio específico de un geopoint concreto
def get_near_employees(geopoint, max_distance):
    nearcomps = pd.DataFrame(list(collection.find(
        {'geopoint': { '$near': {'$geometry': geopoint, '$maxDistance': max_distance}}})))
    return sum(nearcomps['number_of_employees'])

In [9]:
# creamos columna nueva con el número de empleados cerca de cada empresa
for e in distances:
    data['employees{}'.format(e)] = data['geopoint'].apply(lambda x: get_near_employees(x, e))

In [10]:
data.head()

,_id,category_code,city,country_code,founded_year,geopoint,latitude,longitude,name,number_of_employees,type,companies500,companies2000,large500,large2000,employees500,employees2000
0,5cd453a1e74b56f50369c6e3,web,New York,USA,2005,"{'type': 'Point', 'coordinates': [-73.9964312,...",40.723731,-73.996431,Wetpaint,47,startup,2,28,False,False,53,431
1,5cd453a1e74b56f50369c6e4,games_video,San Francisco,USA,2006,"{'type': 'Point', 'coordinates': [-122.409173,...",37.788482,-122.409173,Kyte,40,startup,5,85,False,True,71,2178
2,5cd453a1e74b56f50369c6e5,web,Pleasanton,USA,2005,"{'type': 'Point', 'coordinates': [-121.907768,...",37.697805,-121.907768,Jangl SMS,22,startup,1,2,False,True,22,1622
3,5cd453a1e74b56f50369c6e6,games_video,Los Angeles,USA,2006,"{'type': 'Point', 'coordinates': [-118.254558,...",34.051409,-118.254558,Stickam,35,startup,2,8,False,False,70,167
4,5cd453a1e74b56f50369c6e7,games_video,San Bruno,USA,2005,"{'type': 'Point', 'coordinates': [-122.426804,...",37.627971,-122.426804,YouTube,0,startup,1,1,False,False,0,0


In [11]:
data['large500'].value_counts()

False    1639
True       38
Name: large500, dtype: int64

In [12]:
data.to_csv('tableau_project_pt2.csv')

In [13]:
# TO DO: subir el archivo a Drive desde el programa